In [1]:
import os
import json
import numpy as np

We need to get from every sample:
 - UVW and XY (.new)
 - kalpha 1 and kalpha 2  (.xrdml)
 - 2theta (start and end position) (.xrdml)
 - counts (.xrdml)
 - background (start, step, end) (.bac)

In [2]:
def files_names(path):
    '''Function to see structure of data in files and catalogs'''
    with open("names.txt", "w", encoding="utf-8") as names_f:
        for root, dirs, files in os.walk(path):
            names_f.write(f"[PATH]{root}\n")
            for d in dirs: 
                names_f.write(f"[DIR]{d}\n")
            for f in files: 
                names_f.write(f"[FILE]{f}\n")
            names_f.write("\n")


In [3]:
def document_names(extention,ext_file):
    '''Function separating files with specific extention'''
    new = open(ext_file,'w')
    with open("names.txt",'r') as names_f:
        for line in names_f:
            if '[PATH]' in line:
                new.write(f"{line}")
            elif '[FILE]' and extention in line:
                    new.write(f"{line}")
        new.write(f"\n")

In [4]:
def generate_document(ext,ext_file):
    document_names(ext,ext_file)
    with open(ext_file,'r') as new_file:
        file = new_file.read().replace("\\","/")
    with open(ext_file, "w", encoding="utf-8") as new_file:
        new_file.write(file) 

In [5]:
def read_new_doc(ext_file):
    '''function saving paths and file names in order in variables'''
    path,files_names = [],[]
    with open(ext_file,'r') as new_doc:
        lines = new_doc.readlines()
        for i,line in enumerate(lines):
            if '[PATH]' in line:
                clean_path = line.replace("[PATH]","").strip()
                n = 1
                if i+n< len(lines) and '[FILE]' in lines[i+n]:
                    while '[FILE]' in lines[i+n]:
                        clean_file = lines[i+n].replace("[FILE]","/").strip()
                        full_path = clean_path+clean_file
                        path.append(full_path)
                        file_name = lines[i+n].replace("[FILE]","").strip()
                        files_names.append(file_name)
                        n=n+1
    return path,files_names

In [6]:
def read_inside(path):
    file = open(str(path),'r')
    read_content = file.read()
    print(read_content)
    file.close()

In [7]:
def find_xrdml_file(path):
    '''saving data from files to variables:
        kalpha_theta = ['kAlpha1','kAlpha2','2Theta - start position','2Theta - end position']
        counts = ['all the counts']'''
    base, _ = os.path.splitext(path)
    new_path = base + ".xrdml"
    kalpha1,kalpha2,theta_start,theta_stop,counts = [],[],[],[],[]
    if not os.path.exists(new_path):
        return False, False 
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if '<kAlpha1 unit="Angstrom">' in line:
                kalpha1.append(float(line.split(">")[1].split("<")[0]))
            if '<kAlpha2 unit="Angstrom">' in line:
                kalpha2.append(float(line.split(">")[1].split("<")[0]))
            if '<positions axis="2Theta" unit="deg">' in line:
                theta_start.append(float(numered_lines[i+1].split(">")[1].split("<")[0]))
                theta_stop.append(float(numered_lines[i+2].split(">")[1].split("<")[0]))
            if 'counts'in line:
                count = line.split(">")[1].split("<")[0]
                counts = count.split()
    for i in range(len(counts)):
        counts[i]=int(counts[i])      
    return kalpha1,kalpha2,theta_start,theta_stop,counts

In [8]:
def find_bac_file(path):
    '''saving data from background to variables:
        start_step_end = ['start','step','end']
        counts_bac = ['all the counts from background']'''
    base, _ = os.path.splitext(path)
    print(base)
    new_path = base + ".bac"
    start_step_end,counts_bac = [],[]
    if not os.path.exists(new_path):
        return False, False 
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if i == 0:
                a = line.split()
                for n in range(3):
                    start_step_end.append(float(a[n]))
            if i != 0:
                counts_b = line.split()
                for n in range(len(counts_b)):
                    counts_bac.append(int(counts_b[n]))
    return start_step_end,counts_bac

In [9]:
def find_new_file(path):
    '''function saving uvw and xy
        uvw_xy = ['U','V','W','X','Y','GauSiz','LorSiz','Size-Model']
        and '''
    base, _ = os.path.splitext(path)
    new_path = base + ".new"
    index,uvw_xy = [],[]
    print(base)
    
    if not os.path.exists(new_path):
        return False
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if line.strip() == '!':
                continue
            if '!' in line:
                if i>2:
                    index.append(i)  
        if numered_lines[-2] != '!  2Th1/TOF1    2Th2/TOF2  Pattern to plot\n':
                    index.append(0)  
        fourth_from_end = index[-4]
        uvw_xy = numered_lines[fourth_from_end+1].split()
    for i in range(len(uvw_xy)):
        uvw_xy[i]=float(uvw_xy[i])
    return uvw_xy
    

In [10]:
def change_type(variable,type):
    '''function changing all elements inside variable to specific type'''
    for i in range(len(variable)):
        variable[i] = type(variable[i])
    return variable

In [11]:
def find_sum_file(path):
    '''function saving X and Y parameters after program winplotr fitted the peaks from data 
    and multiply the uncertainties and scor factor to get real uncertainties 
    - x and y is needed to calculate size and strain from lorencian component
    [x, u(x), y, u(y)] '''
    base, _ = os.path.splitext(path)
    print(base)

    new_path = base + ".sum"
    x_y_fitted = []
    if not os.path.exists(new_path):
        return False
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if "=> X and y parameters" in line:
                x_and_ux = line.split()
                y_and_uy = numered_lines[i+1].split()
            if "=> Scor:" in line:
                factor =  line.split()
        factor = factor[-1]
        x_y_fitted.append(x_and_ux[-2])
        x_y_fitted.append(x_and_ux[-1])
        x_y_fitted.append(y_and_uy[0])
        x_y_fitted.append(y_and_uy[1])
    change_type(x_y_fitted,float)
    x_y_fitted[1] = x_y_fitted[1]*float(factor)
    x_y_fitted[3] = x_y_fitted[3]*float(factor)
    return x_y_fitted

In [12]:
def find_hkl_file(path):
    '''function saving  all the data from file to seperate variables and after that - as dictionary'''
    base, _ = os.path.splitext(path)
    new_path = base + ".hkl"
    print(base)
    print(new_path)

    h,k,l,mult,sinT_lamb,tt,fwhm,f2,sf2,hedlines,data,index,json_structure = [],[],[],[],[],[],[],[],[],[],[],[],[]

    if not os.path.exists(new_path):
        return False
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if 'Pattern#' in line: 
                index.append(i)
        index.append(len(numered_lines))

        if len(index)>1:
            for n in range(len(index)-1):
                for i,line in enumerate(numered_lines[index[n]:index[n+1]]):
                    if i>1:
                        if i==2:
                            hedlines.append(line.split())
                        else:
                            data.append(line.split())
                for m in range(index[n+1] - index[n]-3):                    
                    h.append(data[m][0])
                    k.append(data[m][1])
                    l.append(data[m][2])
                    mult.append(data[m][3])
                    sinT_lamb.append(data[m][4])
                    tt.append(data[m][5])
                    fwhm.append(data[m][6])
                    f2.append(data[m][7])
                    sf2.append(data[m][8])
                variables = [h,k,l,mult,sinT_lamb,tt,fwhm,f2,sf2]
                for i in range(len(variables)):
                    if i<4:
                        change_type(variables[i],int)
                    else:
                        change_type(variables[i],float)
                json_structure.append({
                        "h": h,
                        "k": k,
                        "l": l,
                        "mult": mult,
                        "sinT_lamb": sinT_lamb,
                        "tt": tt,
                        "fwhm": fwhm,
                        "f2": f2,
                        "sf2": sf2
                    })
                h,k,l,mult,sinT_lamb,tt,fwhm,f2,sf2,data = [],[],[],[],[],[],[],[],[],[]
        else:
            for i,line in enumerate(numered_lines):
                    if i>1:
                        if i==2:
                            hedlines.append(line.split())
                        else:
                            data.append(line.split())
            for m in range(len(numered_lines)-3):
                h.append(data[m][0])
                k.append(data[m][1])
                l.append(data[m][2])
                mult.append(data[m][3])
                sinT_lamb.append(data[m][4])
                tt.append(data[m][5])
                fwhm.append(data[m][6])
                f2.append(data[m][7])
                sf2.append(data[m][8])
            variables = [h,k,l,mult,sinT_lamb,tt,fwhm,f2,sf2]
            for i in range(len(variables)):
                if i<4:
                    change_type(variables[i],int)
                else:
                    change_type(variables[i],float)
            json_structure = {
                    "h": h,
                    "k": k,
                    "l": l,
                    "mult": mult,
                    "sinT_lamb": sinT_lamb,
                    "tt": tt,
                    "fwhm": fwhm,
                    "f2": f2,
                    "sf2": sf2
                }
    return json_structure
    

In [13]:
def data_storage_save(path,name):  
    ''' function using other function to create a file with all the most important data'''
    only_name, _ = os.path.splitext(name) 
    kalpha1,kalpha2,theta_start,theta_stop,counts = find_xrdml_file(path)
    uvw_xy = find_new_file(path) 
    start_step_end,counts_bac = find_bac_file(path)
    hkl = find_hkl_file(path)
    x_y_fitted = find_sum_file(path)
    counts_str = json.dumps(counts, ensure_ascii=False)
    counts_bac_str = json.dumps(counts_bac, ensure_ascii=False)
    #żeby później wczytać JSON i dostać z powrotem listę, trzeba  użyć json.loads(counts_str).
    all_data = {"name": only_name,"data": {
            "path" : path,
            "kalpha1" : kalpha1,
            "kalpha2" : kalpha2,
            "theta_start" : theta_start, 
            "theta_stop" : theta_stop, 
            "uvw_xy" : uvw_xy, 
            "start_step_end" : start_step_end,
            "counts" : counts_str,
            "counts_bac" : counts_bac_str,  
            "hkl" : hkl,
            "x_y_fitted" : x_y_fitted
        }
    }
    return all_data


In [14]:
path = r"D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane"
# path =r"D:/nai/Nati/studia/LabyNMiT/laby 2 - gondek/gr04/Gr04"
files_names(path)
extentions = ['.xrdml','.new','.bac','hkl','sum']
extentions_files = ['xrdml_files.txt','new_files.txt','bac_files.txt','hkl_files.txt','sum_files.txt']
all_paths,all_names,all_data =[],[],[]
for i in range(len(extentions)):
    generate_document(extentions[i],extentions_files[i])
    paths,names = read_new_doc(extentions_files[i]) 
    all_paths.append(paths)
    all_names.append(names)     
length_names = len(all_names[0][:])           
length_paths = len(all_paths)   

with open('data_storage.json', 'w') as data_storage:
    data_storage.write('')
for i in range(length_names):
    all_data.append(data_storage_save(all_paths[0][i],all_names[0][i]))

# all_data.append(data_storage_save(all_paths[0][0],all_names[0][0]))

with open('data_storage.json', 'a', encoding='utf-8') as f:
    f.write(json.dumps(all_data, ensure_ascii=False, indent=2))
    f.write('\n')
print("File created and filled with data.")

D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Fe3O4_ref
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Fe3O4_ref
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Fe3O4_ref
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Fe3O4_ref.hkl
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Fe3O4_ref
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/M3_Fe3O4
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/M3_Fe3O4
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/M3_Fe3O4
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/M3_Fe3O4.hkl
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/M3_Fe3O4
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Sample_S
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Sample_S
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Sample_S
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Sample_S.hkl
D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Sample_S
D:/nai/Nati/studia/inzynierka/github/X